In [4]:
import numpy as np
from scipy.spatial.distance import cdist, euclidean
import pandas as pd
import matplotlib.pyplot as plt
import copy

import geopandas as gpd
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, LogColorMapper)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()
import pandas as pd
from shapely.geometry import Point, Polygon
import shapely.wkt
import seaborn as sns
from bokeh.models import LinearColorMapper, FixedTicker, ColorBar, NumeralTickFormatter
from bokeh.plotting import figure, output_file, save
from bokeh.io import export_png
from bokeh.plotting import figure, show, output_file
import numpy as np
from bokeh.models import Range1d

Loading BokehJS ...

In [5]:
path_suelo = '/Volumes/cblanesg/Geospatial/datos/suelo_cdmx.csv'
raw_suelo = pd.read_csv(path_suelo)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
espcios_abiertos = raw_suelo[raw_suelo['uso_descripcion'] == 'Espacios Abiertos']

In [22]:
comercios = raw_suelo[raw_suelo['uso_descripcion'] == 'Habitacional con Comercio']
comercios = comercios.sample(frac = .20, replace = True, random_state = 1)

In [27]:
data_comercio = comercios[['Latitud', 'Longitud']]
geosource_comercio = ColumnDataSource(data = dict(x = data_comercio['Longitud'],
                                         y = data_comercio['Latitud']
                                         ))

In [28]:
data_espacio = espcios_abiertos[['Latitud', 'Longitud']]
geosource_espacio = ColumnDataSource(data = dict(x = espcios_abiertos['Longitud'],
                                         y = espcios_abiertos['Latitud']))

In [30]:
p = figure(
           plot_height = 800, plot_width = 800, 
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')

    
p.circle(x='x', y='y',size = 5, color = 'lightblue', source = geosource_comercio)
p.circle(x='x', y='y',size = 5, color = 'lightgreen', source = geosource_espacio)
show(p)

In [81]:
def get_orientation(coord_list):
    '''
    Returns: SDx and SDy
    '''
    lat_lis = [x[0] for x in coord_list]
    long_lis =  [x[1] for x in coord_list]
    
    mean_x = np.mean(long_lis)
    mean_y = np.mean(lat_lis)
    
    sd_x = np.power(np.sum(np.power((long_lis - mean_x), 2))/ len(long_lis), 0.5)
    sd_y = np.power(np.sum(np.power((lat_lis - mean_y), 2))/ len(lat_lis), 0.5)
    
    return(sd_x, sd_y)

In [54]:
class Geometric_Median():
    def __init__(self, array, values_to_check = None, number_of_vals = 5, eps = 1e-5 ):
        '''
        The array is the array of 2-D tuples
        '''
        self.array = array
        self.geometric_median = self.geometric_median(eps)
        self.number_of_vals = number_of_vals
        self.values_to_check = values_to_check
        #self.array_copy = copy.deepcopy(array)

        
        
    def feature_min(self, number_of_vals = 10):
        if self.values_to_check == None:
            self.distances_and_array = self.points_closeby()
            self.values_to_check = self.closest_positions(number_of_vals)
        list_of_distances = []
        for val in self.values_to_check:
            total_distance = np.sum(cdist(self.array, [val]))
            list_of_distances.append(((total_distance), val))
        return(min(list_of_distances))
        
    def geometric_median(self, eps=1e-5):
        '''
        X must be an array of tuples where each value of the tuple corresponds with the
        value in each dimension for that point. 

        For example, if you have 2D points at (0,0), (0, 1), and (1, 1), your 
        input will be np.array([(0, 0), (0, 1), (1, 1)])
        '''
        array = np.array(self.array)
        y = np.mean(array, 0) #use the mean as your starting point

        while True:
            D = cdist(array, [y])    #calculates the distance from each point to point y (the mean)
            nonzeros = (D != 0)[:, 0]  #turns these values to Boolean contitions. If not zero- True

            Dinv = 1 / D[nonzeros]     
            Dinvs = np.sum(Dinv)
            W = Dinv / Dinvs
            T = np.sum(W * array[nonzeros], 0)

            num_zeros = len(array) - np.sum(nonzeros)
            if num_zeros == 0:
                y1 = T
            elif num_zeros == len(array):
                return y
            else:
                R = (T - y) * Dinvs
                r = np.linalg.norm(R)
                rinv = 0 if r == 0 else num_zeros/r
                y1 = max(0, 1-rinv)*T + min(1, rinv)*y

            if euclidean(y, y1) < eps:
                return y1

            y = y1
                        
    def closest_positions(self, number_of_vals ):
        minimum_list = []
        n = 0
        while n < number_of_vals:
            min_value = min(self.distances_and_array)
            minimum_list.append(min_value[1])
            self.distances_and_array.remove(min_value)
            n += 1
        return(minimum_list)


    def points_closeby(self):
        values_from_center = cdist(self.array, [self.geometric_median])
        combination = list(zip(values_from_center, self.array))
        return(combination)

In [82]:
## Data for comercio
coord_list_comercio = []
for lat, long in zip(data_comercio['Latitud'], data_comercio['Longitud']):
    try:
        lat = float(lat)
        long = float(long)
        coord_list_comercio.append((lat, long))
    except:
        continue
lat_lis = [x[0] for x in coord_list_comercio]
long_lis =  [x[1] for x in coord_list_comercio]

geo_med = Geometric_Median(coord_list_comercio)
median_lat_comercio, median_long_comercio = geo_med.geometric_median
sdx_comercio, sdy_comercio = get_orientation(coord_list_comercio)

In [89]:
## Data for espacio
coord_list_espacio = []
for lat, long in zip(data_comercio['Latitud'], data_comercio['Longitud']):
    try:
        lat = float(lat)
        long = float(long)
        coord_list_espacio.append((lat, long))
    except:
        continue
lat_lis_espacio = [x[0] for x in coord_list_espacio]
long_lis_espacio =  [x[1] for x in coord_list_espacio]

geo_med_espacio = Geometric_Median(coord_list_espacio)
median_lat_espacio, median_long_espacio = geo_med_espacio.geometric_median
sdx_espacio, sdy_espacio = get_orientation(coord_list_espacio)
distance, (feature_lat, feature_long) = geo_med_espacio.feature_min()

In [90]:
p = figure(
           plot_height = 800, plot_width = 800, 
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')

    
p.circle(x='x', y='y',size = 5, color = 'lightblue', source = geosource_comercio)
p.circle(x='x', y='y',size = 5, color = 'lightgreen', source = geosource_espacio)


#p.circle([median_long_comercio], [median_lat_comercio], size = 5, color = "navy", alpha=0.5)
#p.circle([median_long_espcio], [median_lat_espacio], size = 5, color = "red", alpha=0.5)

p.ellipse(x = [median_long_comercio], y = [median_lat_comercio], alpha = 0.5,
         width = [sdx_comercio], 
         height = [sdy_comercio],
         color = ['navy'])


p.ellipse(x = [feature_long], y = [feature_lat], alpha = 0.5,
         width = [sdx_espacio], 
         height = [sdy_espacio],
         color = ['red'])



show(p)